In [1]:
import pandas as pd
import numpy as np
import pandas_gbq
from prophet import Prophet
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from google.oauth2.service_account import Credentials
from oauth2client.service_account import ServiceAccountCredentials
from tqdm import tqdm
import time
import pygsheets
import datetime as dt
from datetime import datetime
import time
# Multi-processing
from multiprocessing import Pool, cpu_count
# Disable INFO logging
import logging
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)
import warnings
warnings.filterwarnings("ignore")

In [2]:
credentials = Credentials.from_service_account_file('C:/Users/leemn/OneDrive/Documents/Service Account/cb-prod-297913-d207de781afd.json')

In [3]:
query_drivers = '''
                 SELECT 
*,
SUM(manually_assigned_count+auto_assigned_count) Total_trips,
SUM(manually_assigned_count+auto_assigned_count+none_count+null_count) Total_trips_done,
manually_assigned_count/CASE WHEN SUM(manually_assigned_count+auto_assigned_count) = 0 THEN NULL 
ELSE SUM(manually_assigned_count+auto_assigned_count)
END AS Perc_Manual,
auto_assigned_count/CASE WHEN SUM(manually_assigned_count+auto_assigned_count) = 0 THEN NULL ELSE SUM(manually_assigned_count+auto_assigned_count) 
END AS Perc_Auto,
CASE WHEN driver_avg_daily_trips>branch_avg_daily_trips_per_driver THEN 1
ELSE 0 END AS Exceeded,
CASE WHEN driver_avg_daily_trips>branch_avg_daily_trips_per_driver THEN "Above Average"
ELSE "Below Average" END AS Exceeded2,
FROM
(WITH daily_trips AS (
  SELECT 
    driver.id AS driver_id,
    driver.name AS driver_name,
    branch.name AS branch_name,
    DATE(created_at) AS created_at,
    COUNT(DISTINCT trip_id) AS daily_trips,
    COUNT(DISTINCT order_id) AS daily_orders
  FROM 
    `cb-prod-297913.cb_prod_analytics.cb_end_orders_in_trip`
    WHERE trip_client_id IN ('B0l6eGBH0UB7wOhWzZ9J')
    --AND driver.name IN ('Teboho Jousse')
    --AND branch.name LIKE '%Woolworths Canal Walk%'
  GROUP BY 
    driver_id,
    driver_name,
    branch_name,
    created_at
), branch_avg_daily_trips AS (
  SELECT 
    branch_name,
    AVG(daily_trips.daily_trips) as branch_avg_daily_trips,
    AVG(daily_trips.daily_orders) as branch_avg_daily_orders
  FROM 
    daily_trips
  GROUP BY 
    branch_name
), driver_branch_avg_daily_trips AS (
  SELECT 
    driver_name,
    branch_name,
    created_at,
    AVG(daily_trips.daily_trips) as driver_branch_avg_daily_trips
  FROM 
    daily_trips
  GROUP BY 
    driver_name,
    branch_name,
    created_at
), branch_number_of_drivers AS (
  SELECT 
    branch_name,
    COUNT(DISTINCT driver_id) as number_of_drivers,
  FROM 
    daily_trips
  GROUP BY 
    branch_name

),
driver_assignment_counts AS (
  SELECT
  DATE(created_at) AS created_at,
    driver.name AS driver_name,
    branch.name AS branch_name,
    driver_assignment_type,
    COUNTIF(driver_assignment_type = 'manually_assigned') AS manually_assigned_count,
    COUNTIF(driver_assignment_type = 'auto_assigned') AS auto_assigned_count,
    COUNTIF(driver_assignment_type = 'none') AS none_count,
    COUNTIF(driver_assignment_type IS NULL) AS null_count
  FROM
    `cb-prod-297913.cb_prod_analytics.cb_end_orders_in_trip`
    GROUP BY
    driver_name,
    branch_name,
    driver_assignment_type,
    created_at
)







SELECT 
  daily_trips.created_at,
  daily_trips.driver_name,
  daily_trips.branch_name,
  daily_trips.daily_trips,
  AVG(daily_trips.daily_trips) OVER (PARTITION BY daily_trips.driver_name) AS driver_avg_daily_trips,
  branch_avg_daily_trips.branch_avg_daily_trips AS branch_avg_daily_trips_per_driver ,
  COUNT(DISTINCT daily_trips.created_at) OVER (PARTITION BY daily_trips.driver_name) AS days_worked,
   --AVG(daily_trips.daily_trips) OVER (PARTITION BY daily_trips.driver_name, daily_trips.branch_name) AS daily_trips_per_branch,
  --driver_branch_avg_daily_trips.driver_branch_avg_daily_trips
  daily_trips.daily_orders,
  AVG(daily_trips.daily_orders) OVER (PARTITION BY daily_trips.driver_name) AS driver_avg_daily_orders,
  branch_avg_daily_trips.branch_avg_daily_orders AS branch_avg_daily_orders_per_driver,
  branch_number_of_drivers.number_of_drivers AS number_of_drivers_branch,
  driver_assignment_counts.manually_assigned_count,
  driver_assignment_counts.auto_assigned_count,
  driver_assignment_counts.none_count,
    driver_assignment_counts.null_count,
FROM 
  daily_trips
LEFT JOIN 
  branch_avg_daily_trips 
  ON 
    daily_trips.branch_name = branch_avg_daily_trips.branch_name
LEFT JOIN 
  branch_number_of_drivers 
  ON 
    daily_trips.branch_name = branch_number_of_drivers.branch_name
LEFT JOIN 
  driver_branch_avg_daily_trips 
  ON 
    daily_trips.driver_name = driver_branch_avg_daily_trips.driver_name 
    AND daily_trips.branch_name = driver_branch_avg_daily_trips.branch_name 
    AND daily_trips.created_at = driver_branch_avg_daily_trips.created_at

LEFT JOIN driver_assignment_counts
   ON 
     daily_trips.driver_name = driver_assignment_counts.driver_name 
     AND daily_trips.branch_name = driver_assignment_counts.branch_name
    AND daily_trips.created_at = driver_assignment_counts.created_at 

WHERE daily_trips.branch_name NOT LIKE "%Test%" 
AND daily_trips.branch_name NOT LIKE "%Loop%"
AND daily_trips.branch_name NOT LIKE "%Cowabunga%"
ORDER BY driver_assignment_counts.manually_assigned_count DESC
) x

GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15    
                '''
df_trips = pd.read_gbq(query_drivers, project_id='cb-prod-297913', 
                       dialect='standard', 
                       #progress_bar_type='tqdm_notebook',
                        credentials=credentials)

In [11]:
df_trips['created_at'] = pd.to_datetime(df_trips['created_at'], format='%Y-%m-%d')

In [12]:
# Export to google sheets

file = 'C:/Users/leemn/OneDrive/Documents/Service Account/cb-prod-297913-d207de781afd.json'
sheet_id = '1wgpF2RiV_D16TCuHaLSWY1O3-ZH8uekT-zfgbnFmLMk'
sheet_name = 'branch_driver_2'

def write_to_gsheet(service_file_path, spreadsheet_id, sheet_name, data_df):
    gc = pygsheets.authorize(service_file=service_file_path)
    sh = gc.open_by_key(spreadsheet_id)
    try:
        sh.add_worksheet(sheet_name)
    except:
        pass
    wks_write = sh.worksheet_by_title(sheet_name)
    wks_write.clear('A1',None,'*')
    wks_write.set_dataframe(data_df, (1,1), encoding='utf-8', fit=True)
    wks_write.frozen_rows = 1

write_to_gsheet(file, sheet_id, sheet_name, df_trips)